## Define funções

In [1]:
def download_GOES16_ch13(url, filename):
    import requests
    import warnings
    warnings.filterwarnings("ignore")
    url_filename = f'{url}{filename}'
    myfile = requests.get(url_filename, verify=False)
    open(f'{filename}', 'wb').write(myfile.content)
    return

def netcdf_to_tif(file_netcdf):
    
    # leitura do arquivo NETCDF
    nc_file = xr.open_dataset(file_netcdf)

    # extração da variável de interesse
    temp_ir = nc_file['Band1']

    # transforma de Kelvin para Celsius
    temp_ir = (temp_ir/100.0)-273.15

    # dimensões
    temp_ir = temp_ir.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
    temp_ir.rio.crs

    # define a projeção CRS
    temp_ir.rio.write_crs('epsg: 4326', inplace=True)

    # salva arquivo
    temp_ir.rio.to_raster(f'{file_tif}.tiff')

    return

## Baixa o arquivo netcdf e transforma para tif

In [2]:
# importa biblioteca
import xarray as xr
import leafmap
import rasterio
import rioxarray
import xarray as xr

# Define a data
ano, mes, dia, hor, min = '2024', '05', '02', '04', '00'

# baixando o arquivo NETCDF
url = f'https://ftp.cptec.inpe.br/goes/goes16/retangular/ch13/{ano}/{mes}/'
file_netcdf = f'S10635346_{ano}{mes}{dia}{hor}{min}.nc'
file_tif = f'S10635346_{ano}{mes}{dia}{hor}{min}'
download_GOES16_ch13(url, file_netcdf)

# transforma para TIF
netcdf_to_tif(file_netcdf)

## Plota mapa

In [5]:
# define mapa
Map = leafmap.Map(layers_control=True, zoom=5)

# shapefiles
shp_paises = "https://raw.githubusercontent.com/opengeos/leafmap/master/examples/data/countries.geojson"
Map.add_geojson(shp_paises, layer_name="Paises", style = {"stroke": True,
                                                          "color": 'black',
                                                          "weight": 1.0,
                                                          "opacity": 0.5,
                                                          "fill": False,
                                                          "fillColor": "#3388ff",
                                                          "fillOpacity": 0.1})

shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados Brasileiros", style = {"stroke": True,
                                                                               "color": 'black',
                                                                               "weight": 1.7,
                                                                               "opacity": 0.5,
                                                                               "fill": False,
                                                                               "fillColor": "#3388ff",
                                                                               "fillOpacity": 0.1})
# plota mapa
vmin, vmax = -103.0, 105
Map.add_raster(f'{file_tif}.tiff', 
               indexes=[1], 
               vmin=vmin, 
               vmax=vmax,
               colormap="gray_r", 
               layer_name="Temperatura de Brilho CH13")

# plota texto 
text = f'GOES16 CH13: {ano}-{mes}-{dia} às {hor}{min} UTC'
params = {"fontsize": 20,
          "fontcolor": "blue",
          "bold": True,
          "padding": "2px",
          "background": True,
          "bg_color": "white",
          "border_radius": "5px",
          "position": "topleft"}
Map.add_text(text, **params)

# barra de cores
Map.add_colormap(cmap="gray_r",
                 discrete=True, 
                 label='Temperatura de Brilho ($^o$C)',
                 font_size=8,
                 transparent_bg=True, 
                 position='bottomright',
                 width=3, height=0.4,
                 vmin=vmin, 
                 vmax=vmax)

# exibe o mapa
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [6]:
# salva imagem para HTML
Map.to_html(f"map_goes16_{ano}{mes}{dia}_{hor}{min}.html")